<a href="https://colab.research.google.com/github/Skuerjuan/spotify_chona/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP Spotify
## Felipe Maierowicz
## 4°D

In [1]:
try:
    import pandas as pd
except ImportError:
    print("Pandas not installed")
    %pip install pandas --user

try:
    import matplotlib.pyplot as plt
except ImportError:
    print("Matplotlib not installed")
    %pip install matplotlib --user

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
archivo = "https://raw.githubusercontent.com/IgnacioPardo/Tecnologias_Exponenciales_2025/refs/heads/main/StreamingHistory_Extended.csv"

dataset = pd.read_csv(archivo)
filtro = dataset["is_podcast"] == False
dataset = dataset[filtro] # Borrando los podcasts
dataset.drop(["user_agent_decrypted", "episode_name", "episode_show_name", "spotify_episode_uri", "skipped", "is_podcast"], axis="columns", inplace=True)
# Borrando las columnas relacionadas a los podcasts y skipped se puede sacar de "reason_end"
dataset.head()

<ipython-input-21-df173dfc3ff9>:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(archivo)


,ts,username,platform,ms_played,conn_country,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,reason_start,reason_end,shuffle,offline,offline_timestamp
0,2020-12-06T02:51:11Z,11145402699,"iOS 14.2 (iPhone12,3)",145610,AR,Hey Jude - Remastered 2015,The Beatles,1,spotify:track:0aym2LBJBk9DAYuHHutrIl,trackdone,fwdbtn,True,False,1607222710755
1,2020-12-06T02:53:31Z,11145402699,"iOS 14.2 (iPhone12,3)",138692,AR,Hacer un puente,La Franela,Hacer Un Puente,spotify:track:18Q3WTfyWX1tdS1cp42NhA,fwdbtn,fwdbtn,True,False,1607223071185
2,2020-12-06T02:53:53Z,11145402699,"iOS 14.2 (iPhone12,3)",22662,AR,Happy Together - Remastered,The Turtles,All the Singles,spotify:track:2admbUc576sC6M0QeUkg2C,fwdbtn,fwdbtn,True,False,1607223210387
3,2020-12-06T02:53:56Z,11145402699,"iOS 14.2 (iPhone12,3)",2716,AR,Demons,Imagine Dragons,Night Visions,spotify:track:5qaEfEh1AtSdrdrByCP7qR,fwdbtn,fwdbtn,True,False,1607223233165
4,2020-12-06T02:53:57Z,11145402699,"iOS 14.2 (iPhone12,3)",116,AR,"Paint It, Black",The Rolling Stones,Aftermath,spotify:track:63T7DJ1AFDD6Bn8VzG6JE8,fwdbtn,fwdbtn,True,False,1607223236453


In [23]:
'''
---Análisis de las Columnas:
-shuffle, reason_start, reason_end u offline dependen del
comportamiento del usuario (ms_played depende de la duración de la canción
y de si skippeó o la escuchó entera, que se puede sacar de reason_end)
-columnas de master + spotify_track_uri dependen de la canción y el autor
-ts, offline_timestamp dependen de la fecha y hora de la escucha
-username, platform, conn_country y user_agent dependen
del usuario y el dispositivo
'''
dataset.columns

Index(['ts', 'username', 'platform', 'ms_played', 'conn_country',
       'master_metadata_track_name', 'master_metadata_album_artist_name',
       'master_metadata_album_album_name', 'spotify_track_uri', 'reason_start',
       'reason_end', 'shuffle', 'offline', 'offline_timestamp'],
      dtype='object')

In [8]:
# Primer y última fecha
dataset['ts'] = pd.to_datetime(dataset['ts'])
print(dataset['ts'].min(), " - ", dataset['ts'].max())

2014-06-27 18:01:15+00:00  -  2024-05-23 23:56:20+00:00


In [32]:
# Cantidad de canciones

songs = dataset["spotify_track_uri"]
songs = songs.unique()
len(songs)

11557

In [35]:
# Duración reproducciones

time_spent_song = dataset.groupby('spotify_track_uri')['ms_played'].sum()
time_spent_song.head()

,ms_played
spotify_track_uri,
spotify:track:001b8XFtRhhFrMbWROVgO4,308533
spotify:track:008YILXYfsMa2KuPY8bBtl,169936
spotify:track:009s8MccQSXM4pAQwsP59s,4373
spotify:track:00DiMUseXsW3L8eR8G8xOa,2078
spotify:track:00GOPLxW4PGQuUYdPJh8K1,181406


In [37]:
# Duración por artista

time_spent_artist = dataset.groupby("master_metadata_album_artist_name")['ms_played'].sum()
time_spent_artist.head()

,ms_played
master_metadata_album_artist_name,
VK Blanka,9031
!!!,348
"""Weird Al"" Yankovic",511282
#TocoParaVos,10365206
$NOT,23186


In [50]:
# canción más escuchada
song_uri = time_spent_song.idxmax()
dataset[dataset["spotify_track_uri"] == song_uri][["master_metadata_track_name", "master_metadata_album_artist_name"]]

,master_metadata_track_name,master_metadata_album_artist_name
227,Redbone,Childish Gambino
238,Redbone,Childish Gambino
360,Redbone,Childish Gambino
386,Redbone,Childish Gambino
412,Redbone,Childish Gambino
...,...,...
95895,Redbone,Childish Gambino
96256,Redbone,Childish Gambino
96397,Redbone,Childish Gambino
98600,Redbone,Childish Gambino
